In [31]:
import getpass  # for password input
import psycopg2 # for creting a new db, otherwise jupyter magic %sql will be used
import numpy as np  # for creating random numbers
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # see http://initd.org/psycopg/docs/extensions.html#psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
# start the local posrgres server
command = "/etc/init.d/postgresql start" # command to run from shell use -S as it enables input from stdin
!echo {getpass.getpass()}|sudo -S {command} # run the command using the sudo password

 ········


[sudo] password for fede: Starting postgresql (via systemctl): postgresql.service.


In [17]:
# database test already exists on the server
con = psycopg2.connect(dbname='test',
      user='fede',host='localhost',
      password= getpass.getpass('PostgreSQL password'))
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) # this is to avoid an error `CREATE DATABASE cannot run inside a transaction block`
cur = con.cursor()

PostgreSQL password ········


In [28]:
# list all tables
cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('test',)
('numbers',)
('test2',)
('test3',)
('test4',)


In [29]:
# create table
SQLCOMMAND = """
create table "numbers" (number1 int, number2 int, number3 int);
"""
cur.execute(SQLCOMMAND)

DuplicateTable: relation "numbers" already exists


In [109]:
random_numbers = np.random.randint(0, 100, size=12)
random_numbers.shape = (4, 3)
random_numbers

array([[47, 89, 66],
       [56, 77, 56],
       [94,  8, 89],
       [96, 36, 43]])

In [115]:
print(random_numbers.shape[0])

4


In [121]:
print(random_numbers[0,1])

89


In [126]:
for row in range(random_numbers.shape[0]):
    for col in range(random_numbers.shape[1]):
        # print (row,col)
        print(random_numbers[row][col])

47
89
66
56
77
56
94
8
89
96
36
43


In [46]:
SQLCOMMAND = """
insert into numbers (number1,number2,number3)
values (1,2,3);
"""
cur.execute(sql.SQL(SQLCOMMAND))

In [47]:
cur.execute(sql.SQL('select * from numbers'))
print(cur.fetchall())

[(1.0, 2.0, 3.0), (1.0, 2.0, 3.0)]


In [179]:
# TODO: move to module
SQLCOMMAND: str = 'insert into numbers (number1,number2,number3) VALUES '
# values """
for row in range(random_numbers.shape[0]):
    SQLCOMMAND += f'('
    tempstr = ''
    for col in range(random_numbers.shape[1]):
        tempstr += fr'{random_numbers[row][col]},'
    SQLCOMMAND += tempstr[:-1]
    SQLCOMMAND += f'),'
    # SQLCOMMAND = SQLCOMMAND[:-1]
SQLCOMMAND = SQLCOMMAND[:-1]
print(SQLCOMMAND)

insert into numbers (number1,number2,number3) VALUES (47,89,66),(56,77,56),(94,8,89),(96,36,43)


In [180]:
cur.execute(sql.SQL(SQLCOMMAND))

In [181]:
SQLCOMMAND = """
select * from numbers
"""
cur.execute(SQLCOMMAND)
cur.fetchall()

[(1.0, 2.0, 3.0),
 (1.0, 2.0, 3.0),
 (1.0, None, None),
 (47.0, 89.0, 66.0),
 (56.0, 77.0, 56.0),
 (47.0, 89.0, 66.0),
 (56.0, 77.0, 56.0),
 (94.0, 8.0, 89.0),
 (96.0, 36.0, 43.0)]